In [10]:
!pip install pyarrow pandas numpy boto3

In [11]:

from pathlib import Path

import numpy
import pandas as pd
import requests
from sklearn.model_selection import train_test_split


In [4]:
tmpdir = Path.cwd().parent / "tmp"

filepath = tmpdir / "yellow_tripdata_2024-07.parquet"
if not filepath.is_file():
    endpoint = "https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-07.parquet"
    with requests.get(endpoint) as response:
        response.raise_for_status()
        with filepath.open("wb") as f:
            f.write(response.content)

# Even if file was downloaded, read parquet from it to check it was written correctly
with filepath.open("rb") as f:
    d = pd.read_parquet(f)
assert type(d['PULocationID'][0]) is numpy.int32

In [9]:
d.describe()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee
count,3.076903e+06,3076903,3076903,2.797914e+06,3.076903e+06,2.797914e+06,3.076903e+06,3.076903e+06,3.076903e+06,3.076903e+06,3.076903e+06,3.076903e+06,3.076903e+06,3.076903e+06,3.076903e+06,3.076903e+06,2.797914e+06,2.797914e+06
mean,1.766745e+00,2024-07-17 04:52:59.937344,2024-07-17 05:10:13.448311,1.354542e+00,5.111787e+00,2.581814e+00,1.625984e+02,1.619581e+02,1.137432e+00,1.953443e+01,1.401216e+00,4.776966e-01,3.263495e+00,5.924703e-01,9.547924e-01,2.808619e+01,2.195965e+00,1.678450e-01
min,1.000000e+00,2009-01-01 00:02:24,2009-01-01 00:13:25,0.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,-2.261200e+03,-7.500000e+00,-5.000000e-01,-9.342000e+01,-6.900000e+01,-1.000000e+00,-2.265450e+03,-2.500000e+00,-1.750000e+00
25%,2.000000e+00,2024-07-10 10:55:59,2024-07-10 11:14:03.500000,1.000000e+00,1.040000e+00,1.000000e+00,1.320000e+02,1.130000e+02,1.000000e+00,9.300000e+00,0.000000e+00,5.000000e-01,0.000000e+00,0.000000e+00,1.000000e+00,1.560000e+01,2.500000e+00,0.000000e+00
50%,2.000000e+00,2024-07-17 11:25:26,2024-07-17 11:44:51,1.000000e+00,1.800000e+00,1.000000e+00,1.610000e+02,1.620000e+02,1.000000e+00,1.350000e+01,1.000000e+00,5.000000e-01,2.520000e+00,0.000000e+00,1.000000e+00,2.100000e+01,2.500000e+00,0.000000e+00
75%,2.000000e+00,2024-07-24 13:46:57,2024-07-24 14:05:16.500000,1.000000e+00,3.510000e+00,1.000000e+00,2.310000e+02,2.330000e+02,1.000000e+00,2.260000e+01,2.500000e+00,5.000000e-01,4.200000e+00,0.000000e+00,1.000000e+00,3.061000e+01,2.500000e+00,0.000000e+00
max,2.000000e+00,2024-08-01 23:51:57,2024-08-02 14:48:03,9.000000e+00,3.265055e+05,9.900000e+01,2.650000e+02,2.650000e+02,5.000000e+00,2.261200e+03,1.619000e+01,1.050000e+01,3.000000e+02,1.060000e+02,1.000000e+00,2.265450e+03,2.500000e+00,1.750000e+00
std,4.229035e-01,NaN,NaN,8.377762e-01,4.073898e+02,1.196155e+01,6.378071e+01,6.947267e+01,6.667458e-01,2.061159e+01,1.836983e+00,1.364387e-01,4.141983e+00,2.300883e+00,2.752434e-01,2.532898e+01,9.199493e-01,5.353739e-01


In [10]:
d["tpep_dropoff_datetime"] - d["tpep_pickup_datetime"]

0         0 days 00:11:53
1         0 days 00:39:06
2         0 days 00:06:33
3         0 days 00:16:58
4         0 days 00:26:39
                ...      
3076898   0 days 00:32:00
3076899   0 days 00:20:01
3076900   0 days 00:22:00
3076901   0 days 00:18:06
3076902   0 days 00:18:25
Length: 3076903, dtype: timedelta64[us]

In [11]:
(d["tpep_dropoff_datetime"] - d["tpep_pickup_datetime"]).apply(lambda timediff: timediff.total_seconds())


0           713.0
1          2346.0
2           393.0
3          1018.0
4          1599.0
            ...  
3076898    1920.0
3076899    1201.0
3076900    1320.0
3076901    1086.0
3076902    1105.0
Length: 3076903, dtype: float64

In [12]:
X = pd.DataFrame(index=d.index)
X = X.assign(distance_km=d["trip_distance"])
X

,distance_km
0,3.20
1,19.48
2,1.18
3,9.10
4,17.70
...,...
3076898,5.99
3076899,4.43
3076900,4.80
3076901,3.09


In [13]:
duration_min = (d["tpep_dropoff_datetime"] - d["tpep_pickup_datetime"]).apply(
    lambda timediff: timediff.total_seconds() / 60)
X["duration_min"] = duration_min[
    (duration_min >= 1) & (duration_min <= 60)]  # Only use data for trips lasting max. an hour
X.head()

,distance_km,duration_min
0,3.20,11.883333
1,19.48,39.100000
2,1.18,6.550000
3,9.10,16.966667
4,17.70,26.650000


In [24]:
X = X.assign(start_end_code=(d["PULocationID"].astype(str) + "-" + d["DOLocationID"].astype(str)))
X.head()

,distance_km,duration_sec,start_end_code
0,3.20,713.0,140-79
1,19.48,2346.0,132-113
2,1.18,393.0,237-145
3,9.10,1018.0,138-164
4,17.70,1599.0,132-263


In [25]:
categorical = ["start_end_code"]
numerical = ["distance_km", "duration_min"]

In [24]:
df1 = pd.DataFrame([["a", 1], ["b", 2], ["c", 3], ["d", 4], ["e", 5], ["f", 6], ["g", 7], ["h", 8]],
                   columns=["letter", "number"])
train, test = train_test_split(df1, test_size=0.2, shuffle=False)
display(train)
display(test)

,letter,number
0,a,1
1,b,2
2,c,3
3,d,4
4,e,5
5,f,6


,letter,number
6,g,7
7,h,8
